# Week 4 - Numerical linear algebra

## Solving linear systems

How do we solve

\begin{equation*}
    \left( \begin{array}{cccc} 2 & 0 & 0 & 0 \\
                               1 & 2 & 0 & 1 \\
                               0 & 0 & 2 & -1 \\
                               0 & 1 & -2 & 1 \end{array} \right)
    \left( \begin{array}{c} x_0 \\ x_1 \\ x_2 \\ x_3 \end{array} \right)
    =
    \left( \begin{array}{c} 2 \\ 9 \\ 2 \\ 0 \end{array} \right),
\end{equation*}

for the $x_i$?

In [ ]:
import numpy as np

A = np.array([[2, 0, 0, 0],
              [1, 2, 0, 1],
              [0, 0, 2, -1],
              [0, 1, -2, 1]], dtype=float)
b = np.array([2, 9, 2, 0], dtype=float)

print("A=")
print(A)
print("b=")
print(b)

### Forming an augmented matrix

\begin{equation*}
    \left( \begin{array}{cccc} 2 & 0 & 0 & 0 \\
                               1 & 2 & 0 & 1 \\
                               0 & 0 & 2 & -1 \\
                               0 & 1 & -2 & 1 \end{array} \right)
    \left( \begin{array}{c} x_0 \\ x_1 \\ x_2 \\ x_3 \end{array} \right)
    =
    \left( \begin{array}{c} 2 \\ 9 \\ 2 \\ 0 \end{array} \right),
\end{equation*}
is expressed using the augmented matrix
\begin{equation*}
    \left( \begin{array}{cccc|c} 2 & 0 & 0 & 0 & 2 \\
                                 1 & 2 & 0 & 1 & 9 \\
                                 0 & 0 & 2 & -1 & 2 \\
                                 0 & 1 & -2 & 1 & 0 \end{array} \right).
\end{equation*}

In [ ]:
def augmented_matrix(A, b):
    """Construct an augmented matrix.

    Parameters
    ----------

    A : ndarray
        The left-hand-side matrix.
    b : ndarray
        The right-hand-side vector.

    Returns
    -------

    ndarray
        The augmented matrix.
    """

    N_rows, N_cols = A.shape
    A_aug = np.zeros(shape=(N_rows, N_cols + 1), dtype=float)
    A_aug[:, :-1] = A
    A_aug[:, -1] = b
    return A_aug

In [ ]:
A_aug = augmented_matrix(A, b)
print("A_aug=")
print(f"{A_aug}")

### Elementary row operations

Valid elementary row operations

  - Multiply a row by a non-zero scalar.
  - Add a scalar multiple of one row to another.
  - Swap two rows.

In [ ]:
def scale(A, alpha, i):
    """Multiply a row of a matrix by a factor.

    A : ndarray
        The matrix.
    alpha : float
        The scaling factor.
    i : int
        Defines the row to be scaled.
    """

    A[i, :] *= alpha


def add_multiple(A, alpha, i, j):
    """Add a multiple of one row of a matrix to another row.

    Parameters
    ----------

    A : ndarray
        The matrix.
    alpha : float
        The scaling factor.
    i : int
        Defines the row to be scaled and added.
    j : int
        Defines the row to be added to.
    """

    A[j, :] += alpha * A[i, :]


def swap(A, i, j):
    """Swap two rows in a matrix.
    """

    row_i = A[i, :].copy()
    A[i, :] = A[j, :]
    A[j, :] = row_i

### Gaussian elimination

In [ ]:
def eliminate_sub_diagonal(A, j):
    """Eliminate sub-diagonal elements in a matrix using elementary row
    operations.

    Parameters
    ----------

    A : ndarray
        The matrix.
    j : int
        The column in which to eliminate sub-diagonal elements.
    """

    N_rows = A.shape[0]
    for i in range(j + 1, N_rows):
        add_multiple(A, -A[i, j] / A[j, j], j, i)


def eliminate_super_diagonal(A, j):
    """Eliminate super-diagonal elements in a matrix using elementary row
    operations.

    Parameters
    ----------

    A : ndarray
        The matrix.
    j : int
        The column in which to eliminate super-diagonal elements.
    """

    for i in range(j):
        add_multiple(A, -A[i, j] / A[j, j], j, i)


def normalize_diagonal(A, i):
    """Normalize a diagonal element in a matrix using an elementary row
    operation.


    Parameters
    ----------

    A : ndarray
        The matrix.
    i : int
        The diagonal element to normalize.
    """

    scale(A, 1 / A[i, i], i)

In [ ]:
def gaussian_elimination(A, b):
    """Solve a linear system using Gaussian elimination without pivoting.

    Parameters
    ----------

    A : ndarray
        The left-hand-side matrix.
    b : ndarray
        The right-hand-side vector.

    Returns
    -------

    ndarray
        The solution.
    """

    N_cols = A.shape[1]
    A_aug = augmented_matrix(A, b)
    for j in range(N_cols):
        eliminate_sub_diagonal(A_aug, j)
    for j in range(N_cols - 1, -1, -1):
        normalize_diagonal(A_aug, j)
        eliminate_super_diagonal(A_aug, j)
    x = A_aug[:, -1]
    return x

In [ ]:
x = gaussian_elimination(A, b)
print("x = ")
print(x)

print("A @ x - b")
print(A @ x - b)

print("np.linalg.solve(A, b)")
print(np.linalg.solve(A, b))

### Pivoting

Let's try to solve

\begin{equation*}
    \left( \begin{array}{cc} 0 & 1 \\ 1 & 0 \end{array} \right)
        \left( \begin{array}{c} x_0 \\ x_1 \end{array} \right)
        = \left( \begin{array}{c} -1 \\ -2 \end{array} \right).
\end{equation*}

In [ ]:
A = np.array([[0, 1],
              [1, 0]], dtype=float)
b = np.array([-1, -2], dtype=float)

x = gaussian_elimination(A, b)
print("x = ")
print(x)

But this has the unique solution $x_0 = -2$, $x_1 = -1$? What has gone wrong?

In [ ]:
def gaussian_elimination(A, b):
    """Solve a linear system using Gaussian elimination with partial pivoting.


    Parameters
    ----------

    A : ndarray
        The left-hand-side matrix.
    b : ndarray
        The right-hand-side vector.

    Returns
    -------

    ndarray
        The solution.
    """

    N_rows, N_cols = A.shape
    A_aug = augmented_matrix(A, b)
    for j in range(N_cols):
        i_max = j
        for i in range(j + 1, N_rows):
            if abs(A[i, j]) > abs(A[i_max, j]):
                i_max = i
        if i_max != j:
            swap(A_aug, i_max, j)
        eliminate_sub_diagonal(A_aug, j)
    for j in range(N_cols - 1, -1, -1):
        normalize_diagonal(A_aug, j)
        eliminate_super_diagonal(A_aug, j)
    x = A_aug[:, -1]
    return x

In [ ]:
A = np.array([[0, 1],
              [1, 0]], dtype=float)
b = np.array([-1, -2], dtype=float)

x = gaussian_elimination(A, b)
print("x = ")
print(x)